In [1]:
import numpy as np
import cv2
import os
import dlib
import math
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

In [2]:
#This code cell changes with a given model
emotion_list = ['Happy','Anger','Disgust','Fear','Sad','Surprise','Contempt']
saved_model_file = 'combined6classes(1fold).h5'
recog_model = load_model(saved_model_file)

In [3]:
def predict_emotion(emotion_list, image):
    '''
    Makes predictions on the given image and returns the list of emotions
    '''
    probs = recog_model.predict(image[:,:,:,np.newaxis])
    preds = np.argmax(probs, axis = -1)
    emotion = []
    for i in range(preds.shape[-1]):
        emotion.append(emotion_list[preds[i]])

    return emotion

In [4]:
def rotate_image(img,angle):
    '''
    Returns the rotated image with the given angle
    '''
    rows,cols = img.shape
    M = cv2.getRotationMatrix2D(((rows - 1)/2.0,(cols - 1)/2.0) , angle, 1)
    rot_img = cv2.warpAffine(img,M,(cols,rows))

    return rot_img

In [5]:
def find_angle(point1, point2):
    '''
    To find angle in degrees for the given two points
    '''
    angle_r = math.atan((point2[1] - point1[1])/(point2[0] - point1[0])) # angle in radians
    angle_d = math.degrees(angle_r)                                      # angle in degrees

    return angle_d

In [6]:
def eye_centers(landmarks):
    '''
    To find the eye centers(mean of the 6 landmark points around the eye)
    '''

    point1 = (np.mean(landmarks[36:42,0]),np.mean(landmarks[36:42,1])) #36-41 are landmark points surrounding right eye
    point2 = (np.mean(landmarks[42:48,0]),np.mean(landmarks[42:48,1])) #42-47 are landmark points surrounding left eye

    return point1, point2

In [7]:
def landmark_array(crop_image, face):
    '''
    To convert landmarks which are in dlib rectangle object to an ndarray
    '''

    landmarks = predictor(crop_image, face)
    landmark_array = np.zeros((landmarks.num_parts,2))
    
    for i in range(landmarks.num_parts):
        
        landmark_array[i,0] = landmarks.part(i).x
        landmark_array[i,1] = landmarks.part(i).y

    return landmark_array

In [8]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [9]:
def preprocessing(input_image):
    '''
    The preprocessing involves rotation of the image and image cropping
    Arguments :
    input_image -- array of images of shape (1,h,w)
    Returns :
    List of images after preprocessing
    '''

    preprocessed_faces = []
    faces_coords = []

    for img in input_image:
        
        #To detect all the faces in the image
        faces = detector(img)

        #iterating over each face
        for face in faces:
            
            x1 = face.left() - 100
            y1 = face.top() - 100
            x2 = face.right() + 100
            y2 = face.bottom() + 100
            #following is used to form a rectangle around the face
            faces_coords.append([(x1+100,y1+100),(x2-100,y2-100)])

            #first cropping to reduce unnecessary parts for rotation
            crop_image = img[max(y1,0):min(y2,img.shape[0]),
                             max(x1,0):min(x2,img.shape[1])]

            #As face coords change after cropping
            new_faces = detector(crop_image)
            for new_face in new_faces:
                
                #landmark detection
                landmarks = landmark_array(crop_image, new_face)

                #detect eye cnters
                p1, p2 = eye_centers(landmarks)

                #find angle 
                angle = find_angle(p1, p2)

                #rotate image
                rot_img = rotate_image(crop_image, angle)

                #As face coords change after rotation
                new_faces2 = detector(rot_img)
                for new_face2 in new_faces2:
                    
                    #find length 'd'
                    p1_new, p2_new = eye_centers(landmark_array(rot_img,new_face2))
                    d = cv2.norm(np.array(p1_new) - np.array(p2_new))

                    #mid point of new eye centers
                    d_mid = ((p2_new[0] + p1_new[0])/2.0 , (p2_new[1] + p1_new[1])/2.0)

                    #point above line
                    x_up = d_mid[0]
                    y_up = d_mid[1] - (0.6*d)

                    #cropped image
                    x_start = int(landmarks[0,0])
                    x_end = int(landmarks[16,0])
                    y_start = int(y_up)
                    y_end = int(landmarks[8,1])

                    roi = rot_img[y_start:y_end,x_start:x_end]
                    face_roi = cv2.resize(roi,(256,256))

                    preprocessed_faces.append(face_roi)

    return preprocessed_faces,faces_coords

In [10]:
def normalization(imagedata, mean, std_dev):
    '''
    To apply Histogram equalization and Z-Square Normalization to the preprocessed images

    Arguments :
    imagedata -- array of preprocessed images of shape (m,h,w)
    mean -- mean of imagedata array
    std_dev -- standard deviation of imagedata array

    Returns :
    array of normalized images of shape (m,48,48)
    '''
    normalized_images = []
    for i in range(imagedata.shape[0]):
        
        #Histogram Equilization
        hist_eqv = cv2.equalizeHist(imagedata[i])

        #Z-Square normalization
        zsq_norm = ((hist_eqv - mean)/std_dev)

        #Resize
        resized_image = cv2.resize(zsq_norm, (48,48))
        normalized_images.append(resized_image)

    return np.array(normalized_images)

In [11]:
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX

if not cap.isOpened():
    print('cannot open the camera!')
    exit()

while True:
    #capture frame by frame
    ret, frame = cap.read()
    
    # if frame is read correctly ret is True
    if not ret:
        print("can't recieve the frame.Exiting..")
        break
    
    # operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #To make gray as of shape (m,h,w)
    img = gray.reshape((1,) + gray.shape)

    #Image preprocessing and normalization
    image_list,faces_coords = preprocessing(img)

    #To neglect the case where face is not detected
    if len(image_list) != 0 :
        
        image = np.array(image_list)
        mean, std = image.mean(),image.std()
        image = normalization(image, mean, std)
        emotion = predict_emotion(emotion_list,
                                    image)

        for [(x1,y1),(x2,y2)],i in zip(faces_coords,range(len(emotion))):
            
            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),3)
            cv2.putText(frame,emotion[i],(x1,y1),font,2,(255,255,255),3,
                        cv2.LINE_AA)

        cv2.imshow('Frame',frame) #In colab this changes to cv2_imshow(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()